In [3]:
import pandas as pd
df = pd.read_csv("downloads/pos_100k.csv")

In [4]:
!pip install bertopic
!pip uninstall umap
!pip install umap-learn
!pip install stop-words

     |████████████████████████████████| 57 kB 3.3 MB/s             
     |████████████████████████████████| 13.2 MB 34.8 MB/s            
     |████████████████████████████████| 79 kB 12.6 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 57 kB 1.1 MB/s             
     |████████████████████████████████| 55 kB 7.6 MB/s             
     |████████████████████████████████| 55 kB 7.2 MB/s              
     |████████████████████████████████| 53 kB 4.2 MB/s              
     |████████████████████████████████| 53 kB 3.3 MB/s              
     |████████████████████████████████| 40 kB 1.4 MB/s             
     |████████████████████████████████| 881.9 MB 7.7 kB/s              
     |████████████████████████████████| 88 kB 12.5 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 5.2 MB 67.0 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  

## BERTOPIC

In [5]:
from bertopic import BERTopic
m = BERTopic(embedding_model='sentence-transformers/all-mpnet-base-v2', calculate_probabilities=True, verbose=True)

In [6]:
def extract_topics(texts, distance_thresh=0.035, model=m):
    topic_labels = model.transform(texts)[0]
    topic_dict = {}
    for idx, ID in enumerate(topic_labels):
        topics = model.get_topic(ID)
        for t in topics:
            topic = t[0]
            dist = t[1]
            if dist > distance_thresh:
                if not topic_dict.get(topic):
                    topic_dict[topic] = [ texts[idx] ] 
                else:
                    sents = topic_dict[topic] + [texts[idx]]
                    topic_dict[topic] = sents
    return topic_dict

In [7]:
from stop_words import get_stop_words
def get_topic_name(topic_id, m, take=3,):
    try:
        ts = m.get_topic(topic_id)
        ts = [word[0] for word in ts if word not in get_stop_words('english')]

        name = ""
        for t in ts[:take]:
            name += t
            name += " "
        return name.strip()
    except Exception as e:
        print("Except", topic_id, e)
        return None

In [8]:
# df=df.sample(frac=1)
# df1 = df[40000:]
# sents1= list(df1["text"])

In [9]:
sents= list(df["text"])
import time
#9:50am

In [ ]:
topics, probs = m.fit_transform(sents)
#4:27

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

2022-05-16 17:59:04,838 - BERTopic - Transformed documents to Embeddings
2022-05-16 18:01:08,046 - BERTopic - Reduced dimensionality with UMAP


In [ ]:
topics[:2]

In [ ]:
get_topic_name(22, m, 3)
#tuberculosis tb mycobacterium'

In [ ]:
import numpy as np
df["probs"] = [list(reversed(list(np.argpartition(ps, -3)[-3:]))) for ps in probs]
df["topic_id"] = topics
df["topic_name"] = df1["topic_id"].apply(lambda t: get_topic_name(t,m, 3))
df.to_csv ("downloads/topic200k.csv", index = False, header=True)

In [54]:
df2 = pd.read_csv("downloads/df2.csv")
df0 = pd.read_csv("downloads/df1.csv")

In [56]:
df = df0.append(df1).append(df2)

In [59]:
df.to_csv ("downloads/65k_enriched_topic.csv", index = False, header=True)

In [12]:
import numpy as np
def stepwise(step, df, f, key):
    l = []
    tn=[]
    for idx in range(step, len(df)+step-1, step):
        m = BERTopic(embedding_model='sentence-transformers/all-mpnet-base-v2', calculate_probabilities=True, verbose=True)
        print(idx-step, idx)# dataframe[idx-step:idx])
        sl = df[idx-step:idx][key]
        batch = m.fit_transform(list(sl))
        l = l + list([batch])
#         print(batch[1])
        probs = [list(reversed(list(np.argpartition(ps, -3)[-3:]))) for ps in batch[1]]
        t_names = [get_topic_name(p[0], m, 3) for p in probs]
        tn += [t_names]
    return l, tn

In [15]:
#batches, t_names = stepwise(100, df[:300], m.fit_transform, "text")

In [80]:
def flatten(l):
    return [item for sublist in l for item in sublist]


In [ ]:
t =[]
ps = []
for batch in batches:
    #print(batch[0])
    t += [batch[0]]
    ps += [batch[1]]
#        a+=[t]

a = [flatten(t), flatten(ps)]
len(a), len(a[0])

(2, 66547)

In [ ]:
topics,probs = a

In [ ]:
# td = extract_topics(sentences)

In [94]:
import numpy as np

In [102]:
df["probs"].map(type).unique()

array([<class 'list'>], dtype=object)

In [104]:
m.get_topic()

False

In [101]:

[word[0] for word in m.get_topic(14) if word not in get_stop_words('english')]

['amyotrophic',
 'als',
 'sclerosis',
 'neuron',
 'neurons',
 'neurodegenerative',
 'spinal',
 'degeneration',
 'death',
 'neurone']

In [237]:
df = df.rename(columns={"sentence_id": "problem_id"})

In [2]:
df

,first_sent,doi,coreId,oai,title
0,Liu and Mertz (Genes Dev.,10.1128/MCB.25.15.6303-6313.2005,3717551,oai:pubmedcentral.nih.gov:1190326,Binding of hnRNP L to the Pre-mRNA Processing ...
1,"Rett syndrome (RTT), a neurodevelopmental diso...",10.1086/382228,3710391,oai:pubmedcentral.nih.gov:1182264,X-Chromosome Inactivation Patterns Are Unbalan...
2,Estimating optimal sample size for microbiolog...,10.1128/AEM.69.10.6174-6178.2003,3286476,oai:pubmedcentral.nih.gov:201250,A Model To Estimate the Optimal Sample Size fo...
3,The antiphospholipid syndrome is a thrombotic ...,NaN,8064782,oai:pubmedcentral.nih.gov:2398067,Antiphospholipid syndrome in a patient with ra...
4,Alcohol dependence (AD) is a complex disorder ...,NaN,8674979,oai:pubmedcentral.nih.gov:3266269,Genome-Wide Association Study of Copy Number V...
...,...,...,...,...,...
42196,Autism spectrum disorder (ASD) is a common neu...,NaN,8273706,oai:pubmedcentral.nih.gov:2662567,The genetic and neurobiologic compass points t...
42197,This paper revisits the problem of the readine...,NaN,8689613,oai:pubmedcentral.nih.gov:3291942,On the Relevance of Using Open Wireless Sensor...
42198,Thyroid cancer is a rare disease.,NaN,8126095,oai:pubmedcentral.nih.gov:2466601,Thyroid carcinoma and its treatment
42199,Hyperosmolar hyperglycaemic state (HHS) is a m...,NaN,8696563,oai:pubmedcentral.nih.gov:3302717,A case of hyperosmolar hyperglycaemic state wi...
